<a href="https://colab.research.google.com/github/mjiii25/posco-academy/blob/main/Big-Data/Assignment-1/vitalfew-DT-RF-GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [2]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### **데이터 구성하기**

In [3]:
df_raw = pd.read_excel('/content/mydrive/MyDrive/1. 실습데이터 설명(Data Dictionary).xlsx',
                       sheet_name = 'AIR_POLLUTION')

print(df_raw.shape)
df_raw.head(10)

(366, 15)


,MeasDate,PM10,PM25,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
0,2019-07-01,29.0,20.0,0.054,0.021,0.5,0.003,24.034783,0.00,2.304348,249.130435,63.173913,995.073913,0.0,5.695652
1,2019-07-02,26.0,19.0,0.053,0.020,0.5,0.003,24.287500,0.00,2.258333,264.583333,63.166667,998.600000,0.0,3.833333
2,2019-07-03,30.0,20.0,0.042,0.023,0.4,0.003,24.175000,0.00,1.791667,280.000000,65.333333,998.270833,0.0,6.291667
3,2019-07-04,28.0,17.0,0.034,0.026,0.4,0.003,25.354167,0.00,2.037500,262.500000,58.625000,996.633333,0.0,2.541667
4,2019-07-05,29.0,19.0,0.045,0.035,0.5,0.003,27.295833,0.00,1.454167,175.416667,45.541667,993.537500,0.0,3.916667
5,2019-07-06,29.0,21.0,0.043,0.032,0.5,0.003,29.866667,0.00,1.541667,80.000000,39.708333,993.691667,0.0,3.500000
6,2019-07-07,16.0,12.0,0.043,0.014,0.3,0.002,26.858333,0.00,2.266667,82.916667,43.541667,997.450000,0.0,6.583333
7,2019-07-08,10.0,6.0,0.035,0.019,0.3,0.002,25.125000,0.00,1.962500,75.833333,45.333333,997.941667,0.0,6.583333
8,2019-07-09,10.0,6.0,0.033,0.019,0.4,0.002,25.112500,0.00,1.837500,81.250000,46.250000,996.712500,0.0,4.416667
9,2019-07-10,9.0,6.0,0.027,0.019,0.3,0.002,22.787500,0.82,2.125000,80.000000,61.000000,997.329167,0.0,8.291667


In [4]:
df_raw.drop('PM25', axis = 1, inplace = True)

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   MeasDate   366 non-null    datetime64[ns]
 1   PM10       365 non-null    float64       
 2   O3         365 non-null    float64       
 3   NO2        365 non-null    float64       
 4   CO         311 non-null    float64       
 5   SO2        365 non-null    float64       
 6   TEMP       366 non-null    float64       
 7   RAIN       366 non-null    float64       
 8   WIND       366 non-null    float64       
 9   WIND_DIR   366 non-null    float64       
 10  HUMIDITY   366 non-null    float64       
 11  ATM_PRESS  366 non-null    float64       
 12  SNOW       366 non-null    float64       
 13  CLOUD      366 non-null    float64       
dtypes: datetime64[ns](1), float64(13)
memory usage: 40.2 KB


In [6]:
df_raw.describe().round(3)

,PM10,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
count,365.000,365.000,365.000,311.000,365.000,366.000,366.000,366.000,366.000,366.000,366.000,366.000,366.000
mean,33.422,0.024,0.027,0.517,0.003,13.863,0.382,2.225,209.420,60.292,1005.846,0.023,5.063
std,15.938,0.012,0.010,0.153,0.001,9.830,1.122,0.723,70.753,14.533,8.125,0.222,2.986
min,5.000,0.002,0.008,0.300,0.002,-7.954,0.000,0.942,50.000,17.875,983.800,0.000,0.000
25%,22.000,0.014,0.019,0.400,0.003,5.493,0.000,1.722,160.000,49.677,999.423,0.000,2.510
50%,31.000,0.023,0.025,0.500,0.003,13.996,0.000,2.094,221.042,61.062,1006.408,0.000,5.312
75%,42.000,0.032,0.034,0.600,0.003,23.073,0.115,2.620,265.833,69.917,1011.571,0.000,7.573
max,101.000,0.062,0.059,1.000,0.005,31.721,11.800,5.883,351.250,94.958,1025.529,3.188,10.000


**결측값 확인하기**

In [7]:
df_raw.isnull().sum()

MeasDate      0
PM10          1
O3            1
NO2           1
CO           55
SO2           1
TEMP          0
RAIN          0
WIND          0
WIND_DIR      0
HUMIDITY      0
ATM_PRESS     0
SNOW          0
CLOUD         0
dtype: int64

In [8]:
df_raw.fillna(df_raw.median(), inplace = True)

df_raw.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


,MeasDate,PM10,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
0,2019-07-01,29.0,0.054,0.021,0.5,0.003,24.034783,0.0,2.304348,249.130435,63.173913,995.073913,0.0,5.695652
1,2019-07-02,26.0,0.053,0.020,0.5,0.003,24.287500,0.0,2.258333,264.583333,63.166667,998.600000,0.0,3.833333
2,2019-07-03,30.0,0.042,0.023,0.4,0.003,24.175000,0.0,1.791667,280.000000,65.333333,998.270833,0.0,6.291667
3,2019-07-04,28.0,0.034,0.026,0.4,0.003,25.354167,0.0,2.037500,262.500000,58.625000,996.633333,0.0,2.541667
4,2019-07-05,29.0,0.045,0.035,0.5,0.003,27.295833,0.0,1.454167,175.416667,45.541667,993.537500,0.0,3.916667


In [9]:
df_raw.isnull().sum()

MeasDate     0
PM10         0
O3           0
NO2          0
CO           0
SO2          0
TEMP         0
RAIN         0
WIND         0
WIND_DIR     0
HUMIDITY     0
ATM_PRESS    0
SNOW         0
CLOUD        0
dtype: int64

**데이터 분할하기**

In [10]:
df_raw_x = df_raw.drop(['MeasDate', 'PM10'], axis = 1, inplace = False)
df_raw_y = df_raw['PM10']

In [11]:
print(df_raw_x.shape)
df_raw_x.head()

(366, 12)


,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
0,0.054,0.021,0.5,0.003,24.034783,0.0,2.304348,249.130435,63.173913,995.073913,0.0,5.695652
1,0.053,0.020,0.5,0.003,24.287500,0.0,2.258333,264.583333,63.166667,998.600000,0.0,3.833333
2,0.042,0.023,0.4,0.003,24.175000,0.0,1.791667,280.000000,65.333333,998.270833,0.0,6.291667
3,0.034,0.026,0.4,0.003,25.354167,0.0,2.037500,262.500000,58.625000,996.633333,0.0,2.541667
4,0.045,0.035,0.5,0.003,27.295833,0.0,1.454167,175.416667,45.541667,993.537500,0.0,3.916667


In [12]:
print(df_raw_y.shape)
df_raw_y.head()

(366,)


0    29.0
1    26.0
2    30.0
3    28.0
4    29.0
Name: PM10, dtype: float64

In [13]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x,
                                                                df_raw_y,
                                                                test_size = 0.3)

print('Shape of df_train_x : {}'.format(df_train_x.shape))
print('Shape of df_test_x : {}'.format(df_test_x.shape))
print('Shape of df_train_y : {}'.format(df_train_y.shape))
print('Shape of df_test_y : {}'.format(df_test_y.shape))

Shape of df_train_x : (256, 12)
Shape of df_test_x : (110, 12)
Shape of df_train_y : (256,)
Shape of df_test_y : (110,)


In [14]:
print(df_train_x.shape)
df_train_x.head()

(256, 12)


,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
267,0.024,0.033,0.5,0.003,11.312500,0.000000,1.862500,189.583333,37.083333,1009.604167,0.0,0.125000
84,0.032,0.017,0.3,0.002,20.108333,0.000000,2.304167,274.166667,50.125000,1003.795833,0.0,4.375000
340,0.037,0.020,0.4,0.002,22.862500,0.000000,1.962500,270.000000,71.458333,997.050000,0.0,5.958333
313,0.024,0.017,0.4,0.002,14.333333,1.016667,3.558333,135.833333,93.166667,995.579167,0.0,10.000000
136,0.021,0.021,0.4,0.003,1.520833,0.000000,3.091667,252.083333,30.416667,1013.633333,0.0,0.541667


In [15]:
print(df_train_y.shape)
df_train_y.head()

(256,)


267    52.0
84     12.0
340    39.0
313    14.0
136    18.0
Name: PM10, dtype: float64

In [16]:
print(df_test_x.shape)
df_test_x.head()

(110, 12)


,O3,NO2,CO,SO2,TEMP,RAIN,WIND,WIND_DIR,HUMIDITY,ATM_PRESS,SNOW,CLOUD
155,0.007,0.039,0.6,0.004,-0.025000,0.000000,1.466667,181.666667,58.083333,1014.408333,0.00,5.583333
101,0.011,0.040,0.5,0.003,14.779167,0.000000,1.266667,279.583333,60.333333,1006.450000,0.00,8.583333
309,0.022,0.025,0.5,0.003,16.183333,0.000000,2.012500,184.583333,67.625000,1001.445833,0.00,5.416667
218,0.019,0.024,0.5,0.003,-1.416667,0.566667,2.858333,225.833333,52.250000,1015.779167,0.62,4.083333
87,0.013,0.042,0.6,0.003,22.729167,0.000000,1.708333,259.166667,59.625000,1013.495833,0.00,8.125000


In [17]:
print(df_test_y.shape)
df_test_y.head()

(110,)


155    26.0
101    26.0
309    29.0
218    23.0
87     27.0
Name: PM10, dtype: float64

### **변수 중요도 확인하기**

#### **의사결정나무(Decision Tree)**

In [18]:
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(dt_regressor.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(dt_regressor.score(df_test_x, df_test_y)))

Model Score on train set : 1.000
Model Score on test set : -0.269


In [19]:
dt_importance = pd.DataFrame()
dt_importance['Feature'] = df_train_x.columns
dt_importance['Importance'] = dt_regressor.feature_importances_

dt_importance.sort_values('Importance', ascending = False, inplace = True)
dt_importance.round(3).head()

,Feature,Importance
2,CO,0.450
11,CLOUD,0.102
9,ATM_PRESS,0.089
0,O3,0.081
4,TEMP,0.079


**<모델 성능>**
- 기본 의사결정나무는 train과 test 데이터에 대해 각각 100%, 7.8%의 성능을 보여주고 있다. 
- Train 데이터에 있어서 매우 과대적합인 것으로 확인되며, 이는 모델의 성능이 매우 좋지 않다는 점을 의미한다.


**<변수 중요도>**
- 의사결정나무 모델을 통해 변수 중요도를 추출해보았다. 
- CO가 0.450, CLOUD가 0.102, ATM_PRES가 0.089, O3이 0.081, 그리고 TEMP가 0.079의 변수 중요도를 보여주고 있다.

#### **랜덤 포레스트 (Random Forest)**

In [20]:
rf_regressor = RandomForestRegressor()
rf_regressor.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(rf_regressor.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(rf_regressor.score(df_test_x, df_test_y)))

Model Score on train set : 0.921
Model Score on test set : 0.381


In [21]:
rf_importance = pd.DataFrame()
rf_importance['Feature'] = df_train_x.columns
rf_importance['Importance'] = rf_regressor.feature_importances_

rf_importance.sort_values('Importance', ascending = False, inplace = True)
rf_importance.round(3).head()

,Feature,Importance
2,CO,0.388
0,O3,0.103
4,TEMP,0.086
11,CLOUD,0.082
1,NO2,0.069


**<모델 성능>**
- 기본 랜덤포레스트 예측 모델은 train과 test 데이터에 대해 각각 92.1%, 38.1%의 설명력을 갖는다.
- 의사결정나무에 비해서는 과대적합의 경향성이 적어보이지만, 여전히 모델의 일반화 가능성은 높아 보이지 않는다.

**<변수 중요도>**
- CO가 0.388, O3이 0.013, TEMP가 0.086, CLOUD가 0.082, 그리고 NO2가 0.069의 변수 중요도를 가진다.

#### **그래디언트 부스팅 (Gradient Boosting)**

In [22]:
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(gb_regressor.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(gb_regressor.score(df_test_x, df_test_y)))

Model Score on train set : 0.933
Model Score on test set : 0.364


In [23]:
gb_importance = pd.DataFrame()
gb_importance['Feature'] = df_train_x.columns
gb_importance['Importance'] = gb_regressor.feature_importances_

gb_importance.sort_values('Importance', ascending = False, inplace = True)
gb_importance.round(3).head()

,Feature,Importance
2,CO,0.429
0,O3,0.130
4,TEMP,0.119
11,CLOUD,0.084
7,WIND_DIR,0.065


**<모델 성능>**
- 기본 그래디언트 부스팅 모델은 train과 test 데이터에 대해 각각 93.3%, 36.4%의 설명력을 갖는다.
- 의사결정나무에 비해서 모델의 성능이 양호한 것으로 판단되나, 여전히 과대적합의 문제점을 가지고 있다.

**<변수 중요도>**
- CO가 0.429, O3이 0.130, TEMP가 0.119, CLOUD가 0.084, 그리고 WIND_DIR가 0.065의 변수 중요도를 갖는다.

### **결론**

- 의사결정나무, 랜덤 포레스트, 그리고 그래디언트 부스팅 예측기를 이용하여 추출한 변수 중요도는 다음과 같다.


| 모델명/순위 |  의사결정나무  |  랜덤 포레스트   |  그래디언트 부스팅  |
|------------|:--------------:|:----------------:|:------------------:|
|   1    |CO (0.450)          | CO (0.388)       |CO (0.429)          |
|   2    |CLOUD (0.102)       | O3 (0.103)       | O3 (0.130)         |
|   3    |ATM_PRESS (0.089)   |TEMP (0.086)      |TEMP (0.119)        |
|   4    |O3 (0.081)          | CLOUD (0.082)    |CLOUD (0.084)       |
|   5    |TEMP (0.079)        |NO2 (0.069)       |WIND_DIR (0.065)    |

* 3개의 예측 모델 모두 중요하게 고려한 변수는 CO, O3, TEMP, CLOUD이다.
* 일산화탄소 농도가 미세먼지 발생에 압도적인 영향을 미치며, 오존 농도, 기온, 전운량 또한 미세먼지를 유발하는 주요한 원인이라는 점을 파악할 수 있다.